In [1]:
from pathlib import Path
import sqlalchemy
import pandas as pd
from utils import * 

In [2]:
queries_dir = Path('queries/')
enum_query = read_sql(queries_dir / "1.enum_query.sql")
table_query = read_sql(queries_dir / "2.table_query.sql")

datasets_dir = Path("../datasets/")
haccp_dataset_path = datasets_dir / "haccp_dataset_200525221016.csv"

In [3]:
conn = connect()

In [4]:
initializ_db(conn)

In [5]:
execute_queries(conn, enum_query)
execute_queries(conn, table_query)

In [7]:
haccp_dataset.head()

NameError: name 'haccp_dataset' is not defined

In [8]:
haccp_dataset = pd.read_csv(haccp_dataset_path)
haccp_dataset = haccp_dataset.drop(['rnum', 'productgb', 'prdkindstate'], axis=1)

In [9]:
replace_dict = {
    '(' : ' ',
    ')' : ' ',
    '%' : '%%',
    "'" : '',
    '\n' : ' ',
    '\r' : ' '
}
for data in haccp_dataset.values:
    new_data = []
    for s in data:
        if type(s) == str:
            for org, dst in replace_dict.items():
                s = s.replace(org, dst)
        new_data.append(s)
    reportno, prdnm, rawmtrl, allergy, nutrient, barcode, prdkind, manu, seller, capa, img1, img2 = new_data
    val = "'{reportno}', '{prdnm}', '{rawmtrl}', '{allergy}', '{nutrient}','{barcode}',\
    '{prdkind}', '{manu}', '{seller}', '{capa}', '{img1}', '{img2}'".format(reportno = reportno, prdnm = prdnm, rawmtrl = rawmtrl,
                                                                 allergy = allergy, nutrient = nutrient, barcode = barcode,
                                                                 prdkind = prdkind, manu=manu, seller=seller, capa=capa, img1=img1, img2=img2)
    val = val.replace('(', ' ').replace(')', ' ').replace('%', '%%')
    insert_query = "INSERT INTO foodinfo VALUES({})".format(val)
    execute_query(conn, insert_query)